## Using `PEPit` to verify the lower-bound on the NCGPEP contraction factor

In [1]:
import PEPit

In [2]:
from PEPit import *
from PEPit.functions import SmoothStronglyConvexFunction

In [3]:
def NCG_PEP(L, mu, gamma, beta, c0, eta, n, rho_LB, rho_UB,  verbose=1):
    
    # Instantiate PEP
    problem = PEP()
    
    # Declare a smooth strongly convex function
    f = problem.declare_function(SmoothStronglyConvexFunction, mu=mu, L=L)
    
    # Start by defining its unique optimal point xs = x_* and 
    # corresponding function value fs = f_*
    xs = f.stationary_point()
    fs = f.value(xs)
    
    # Then define the starting point x0 of the algorithm as well as corresponding gradient and function value g0 and f0
    x0 = problem.set_initial_point()
    g0, f0 = f.oracle(x0)
    
    d0 = Point()
    d_new = d0
    x_new = x0
    g_new = g0
    
    problem.add_constraint((d0) ** 2 <= c0*(g0) ** 2)

    problem.add_constraint((d0) ** 2 >= (g0) ** 2)

    problem.add_constraint(g0 * d0 == (g0) ** 2)
    
    # Set the initial constraint that is the difference between f0 and f_*
    problem.set_initial_condition(f0 - fs == 1)
    
    for i in range(n):
        print(i)
        d_old = d_new
        x_old = x_new
        g_old = g_new
        x_new = x_old - gamma[i] * d_old
        g_new = f.gradient(x_new)
        d_new = g_new + beta[i] * d_old
        problem.add_constraint(g_new * d_old == 0)
        problem.add_constraint(g_new * (x_old-x_new) == 0)
        problem.add_constraint(g_old * d_old == g_old ** 2)
        if i < n-1:
            print('Adding beta[i] constraint')
            problem.add_constraint(beta[i] * g_old ** 2 == g_new **2 - eta * (g_new * g_old))
        
      
    # Set the performance metric to the function value accuracy
    problem.set_performance_metric(f.value(x_new) - fs)   
    
    # Solve the PEP
    verbose = 1
    pepit_verbose = max(verbose, 0)
    pepit_rho = problem.solve(verbose=pepit_verbose)
    
    
    # Print conclusion if required
    if verbose != -1:
        print('*** Example file ***')
        print("\tPEPit guarantee: (Lower bound)\t\t\t f(x_",n,")-f_* >= {:.6} (f(x_0)-f_*)".format(pepit_rho))
        print("\tNCG-PEP guarantee: (Lower bound)\t\t\t f(x_",n,")-f_* >= {:.6} (f(x_0)-f_*)".format(rho_LB))
        print("\tNCG-PEP guarantee: (Upper bound)\t\t\t f(x_",n,")-f_* <= {:.6} (f(x_0)-f_*)".format(rho_UB))
        
    return pepit_rho   

In [4]:
## Parameter values
L = 1;
eta = 1; # eta = 1 means PRP, eta = 0 means FR

## Get the following from the file "Using_the_datasets_Julia.ipynb"

mu = 0.5
n = 2
beta = [0.11015521120443346, 0.0]
gamma = [1.3127063659282785, 1.1954627659433608]
rho_LB = 0.056103782765843734
rho_UB = 0.056124266554459026

# factor for PRP
c0 = (L+mu)**2/(4*mu*L) 

In [5]:
rho_PEP = NCG_PEP(L, mu, gamma, beta, c0, eta, n, rho_LB, rho_UB,  verbose=1)

0
Adding beta[i] constraint
1
(PEPit) Setting up the problem: size of the main PSD matrix: 6x6
(PEPit) Setting up the problem: performance measure is minimum of 1 element(s)
(PEPit) Setting up the problem: Adding initial conditions and general constraints ...
(PEPit) Setting up the problem: initial conditions and general constraints (11 constraint(s) added)
(PEPit) Setting up the problem: interpolation conditions for 1 function(s)
		 function 1 : Adding 12 scalar constraint(s) ...
		 function 1 : 12 scalar constraint(s) added
(PEPit) Compiling SDP
(PEPit) Calling SDP solver
(PEPit) Solver status: optimal (solver: MOSEK); optimal value: 0.05610383801673924
(PEPit) Postprocessing: solver's output is not entirely feasible (smallest eigenvalue of the Gram matrix is: -2.47e-08 < 0).
 Small deviation from 0 may simply be due to numerical error. Big ones should be deeply investigated.
 In any case, from now the provided values of parameters are based on the projection of the Gram matrix onto 